In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
import os
import cv2
from PIL import Image
from io import BytesIO

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "MIG-d456bb71-756c-5f6b-b13b-6793b90de8b7"

2023-05-20 12:14:10.797791: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 12:14:11.561013: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bigdata_user5/Desktop/envs/nobo/lib/
2023-05-20 12:14:11.561089: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/bigdata_user5/Desktop/envs/nobo/lib/
2023-05-20 12:14:11.561095: W tensorflow/com

In [3]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
train_dataset_path = os.listdir('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down')

train_subject_types = os.listdir('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down')

input_images=[]
input_labels=[]

train_type = ['middlefinger']
train_set=['set1','set2','set3','set4','set5','set6']

def scan_folder(train_dataset_path,label):
    images = []
    labels=[]
    complete_path=[x[0] for x in os.walk(train_dataset_path)]
    for file_name in complete_path:
        #print(file_name)
        for fn in  os.listdir(file_name) :
            # print(fn)
            if fn.endswith(".bmp"):
            # if it's a txt file, print its name (or do whatever you want)
                # print(file_name)
                #im = Image.open(file_name+'/'+fn)
                im = cv2.imread(file_name+'/'+fn)
                # im.show()
                #img_arr=cv2.resize(im,(160,160))
                img_arr=im 
                images.append(img_arr)
                labels.append(label)
                #print(im.size)
                #print(im.filename)
             
        
        else:
            current_path = "".join((train_dataset_path, "/", file_name))
            if os.path.isdir(current_path):
                # if we're checking a sub-directory, recursively call this method
                scan_folder(current_path)
    #print(images)
    return images,labels


for item1 in train_subject_types:
    all_train_subjects = os.listdir('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down' + '/' +item1)
    #print(all_subjects)
    for item2 in train_type:
        for item3 in train_set:
            path=str('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down'  + '/' +item1 +  '/session1'+ '/' +item2 + '/'+item3+ '/3D')
            img,label=scan_folder(path,item1)
            input_images.extend(img)
            input_labels.extend(label)
            #print(input_labels)

input_train_images=np.array(input_images)  
input_train_labels=np.array(input_labels)
print(len(input_train_images))
print(len(input_train_labels))
#print(input_train_images)

7980
7980


In [5]:
test_dataset_path = os.listdir('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down')

test_subject_types = os.listdir('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down')

input_images=[]
input_labels=[]

test_type = ['middlefinger']
test_set=['set1','set2','set3','set4','set5','set6']


def scan_folder(test_dataset_path,label):
    images = []
    labels=[]
    #print(test_dataset_path)
    complete_path=[x[0] for x in os.walk(test_dataset_path)]
    #print(complete_path)
    for file_name in complete_path:
        #print(file_name)
        for fn in  os.listdir(file_name) :
            # print(fn)
            if fn.endswith(".bmp"):
            # if it's a txt file, print its name (or do whatever you want)
                # print(file_name)
                #im = Image.open(file_name+'/'+fn)
                im = cv2.imread(file_name+'/'+fn)
                # im.show()
                img_arr=cv2.resize(im,(212,149))
                #img_arr=im
                images.append(img_arr)
                labels.append(label)
             
        
        else:
            current_path = "".join((test_dataset_path, "/", file_name))
            if os.path.isdir(current_path):
                # if we're checking a sub-directory, recursively call this method
                scan_folder(current_path)
    #print(images)
    return images,labels


for item1 in train_subject_types:
    #all_train_subjects = os.listdir('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down' + '/' +item1)
    #print(all_train_subjects)
    for item2 in test_type:
        for item3 in test_set:
            path=str('data/3Dfingerknuckle/3D_Finger_Knuckle_Database/segmented_data_down'  + '/' +item1 +  '/session2'+ '/' +item2 + '/'+item3+ '/3D')
            img,label=scan_folder(path,item1)
            input_images.extend(img)
            input_labels.extend(label)
            # print(input_labels)

input_test_images=np.array(input_images) #,axis=0)
input_test_labels=np.array(input_labels)
# print(input_labels.shape)
# input_test_labels=np.concatenate(np.array(input_labels),axis=0)
print(input_test_images.shape)
print(input_test_labels.shape)
print(len(input_test_images))    
print(len(input_test_labels))

(7980, 149, 212, 3)
(7980,)
7980
7980


In [6]:
X_train = input_train_images
Y_train = input_train_labels

X_test = input_test_images
Y_test = input_test_labels

print(X_train.shape)
print(Y_train.shape)
print(X_test.shape)
print(Y_test.shape)

(7980, 149, 212, 3)
(7980,)
(7980, 149, 212, 3)
(7980,)


In [7]:
# X_train = X_train/22.0
# X_test = X_test/35.0

# X_train[1,:]
# X_test[1,:]

from sklearn.preprocessing import LabelEncoder
lb= LabelEncoder()
Y_train = lb.fit_transform(Y_train)
Y_test = lb.fit_transform(Y_test)

In [8]:
import tensorflow as tf
print(tf.__version__)
print(tf.test.is_gpu_available())

2.11.0
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.
True


2023-05-20 12:14:49.761050: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-20 12:14:50.304426: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /device:GPU:0 with 38541 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 3g.40gb, pci bus id: 0000:c2:00.0, compute capability: 8.0


In [9]:
from keras.layers import Dense

img_height,img_width = 149,212 
#If imagenet weights are being loaded, 
#input must have a static square shape (one of (128, 128), (160, 160), (192, 192), or (224, 224))
base_model = tf.keras.applications.DenseNet121(weights="imagenet", include_top=False, input_shape= (img_height,img_width,3))

x = base_model.output
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.5)(x)
predictions = Dense(190, activation= 'softmax')(x)
model = tf.keras.Model(inputs = base_model.input, outputs = predictions)

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

2023-05-20 12:14:51.735929: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38541 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB MIG 3g.40gb, pci bus id: 0000:c2:00.0, compute capability: 8.0


In [10]:
history = model.fit(X_train, Y_train, epochs=5, batch_size=128)

Epoch 1/5


2023-05-20 12:15:31.338371: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-05-20 12:15:34.279975: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-05-20 12:15:34.282403: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x4ae06270 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-05-20 12:15:34.282435: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB MIG 3g.40gb, Compute Capability 8.0
2023-05-20 12:15:34.287530: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-05-20 12:15:34.400317: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifeti

63/63 [==============================] - 76s 327ms/step - loss: 2.7716 - accuracy: 0.4504
Epoch 2/5
63/63 [==============================] - 18s 293ms/step - loss: 0.0903 - accuracy: 0.9907
Epoch 3/5
63/63 [==============================] - 18s 291ms/step - loss: 0.0110 - accuracy: 1.0000
Epoch 4/5
63/63 [==============================] - 18s 290ms/step - loss: 0.0055 - accuracy: 1.0000
Epoch 5/5
63/63 [==============================] - 18s 290ms/step - loss: 0.0041 - accuracy: 1.0000


In [11]:
model.evaluate(X_test, Y_test, batch_size=128)

63/63 [==============================] - 7s 78ms/step - loss: 2.5812 - accuracy: 0.4752


[2.5812292098999023, 0.4751879572868347]

In [12]:
model.save('model/densenet121.hdf5')

In [13]:
np.save('model/densenet_training_history.npy', history.history)

In [14]:
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
import matplotlib.pyplot as plt

y_pred = model.predict(X_test)

y_pred = np.argmax(y_pred, axis=1)

cm= confusion_matrix(Y_test,y_pred)
# np.set_printoptions(precision=2)
cr = classification_report(Y_test, y_pred, digits=4)
print(cr)
print(cm)

250/250 [==============================] - 9s 27ms/step
              precision    recall  f1-score   support

           0     1.0000    0.0476    0.0909        42
           1     1.0000    0.0238    0.0465        42
           2     0.3253    0.6429    0.4320        42
           3     1.0000    0.3810    0.5517        42
           4     0.5455    1.0000    0.7059        42
           5     0.5753    1.0000    0.7304        42
           6     0.8000    0.9524    0.8696        42
           7     0.2143    0.4286    0.2857        42
           8     0.8235    1.0000    0.9032        42
           9     0.2718    0.6667    0.3862        42
          10     0.8750    1.0000    0.9333        42
          11     0.0741    0.0476    0.0580        42
          12     0.0000    0.0000    0.0000        42
          13     0.8667    0.9286    0.8966        42
          14     0.4516    1.0000    0.6222        42
          15     0.4200    1.0000    0.5915        42
          16     0.7368  

/home/bigdata_user5/Desktop/envs/nobo/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bigdata_user5/Desktop/envs/nobo/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/bigdata_user5/Desktop/envs/nobo/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,

In [15]:
import numpy as np
from sklearn.metrics import confusion_matrix

# assume y_true is the true labels and y_pred is the predicted labels
cm = confusion_matrix(Y_test, y_pred)

# calculate total true positives and false negatives
tp = np.diag(cm)
fn = np.sum(cm, axis=1) - tp

# calculate total sensitivity (TPR)
sensitivity = np.sum(tp) / (np.sum(tp) + np.sum(fn))

print("Sensitivity (TPR) of the model:", sensitivity)

Sensitivity (TPR) of the model: 0.47518796992481205


In [17]:
import numpy as np
from sklearn.metrics import confusion_matrix

# assume y_true is the true labels and y_pred is the predicted labels
cm = confusion_matrix(Y_test, y_pred)

# calculate false positives (FP), false negatives (FN), true positives (TP), and true negatives (TN)
FP = np.sum(cm, axis=0) - np.diag(cm)
FN = np.sum(cm, axis=1) - np.diag(cm)
TP = np.diag(cm)
TN = np.sum(cm) - (FP + FN + TP)

# calculate the False Positive Rate (FPR) and False Negative Rate (FNR)
FPR = FP / (FP + TN)
FNR = FN / (FN + TP)

# calculate the Crossover Error Rate (CER)
CER = (FPR + FNR) / 2

print("Crossover Error Rate (CER):", np.mean(CER))

FPR = np.sum(FP) / np.sum(FP + TN)
FNR = np.sum(FN) / np.sum(FN + TP)
FDR = np.sum(FP) / np.sum(FP + TP)

print("false positive rate for the model:", FPR)
print("false negative rate for the model:", FNR)
print(FDR)

Crossover Error Rate (CER): 0.26379440665154946
false positive rate for the model: 0.0027767832279110474
false negative rate for the model: 0.524812030075188
0.524812030075188
